In [ ]:
		  ,[caFICO_Binned]	=	CASE
												WHEN [FICO] = 0		THEN '00. Missing'
												WHEN NOT([FICO] between 300 and 850) OR [FICO] IS NULL THEN '00. Missing'
												WHEN [FICO] <= 500	THEN '01. 500-'
												WHEN [FICO] <= 560	THEN '02. LE 560'
												WHEN [FICO] <= 580	THEN '03. LE 580'
												WHEN [FICO] <= 600	THEN '04. LE 600'
												WHEN [FICO] <= 620	THEN '05. LE 620'
												WHEN [FICO] <= 640	THEN '06. LE 640'
												WHEN [FICO] <= 660	THEN '07. LE 660'
												WHEN [FICO] <= 680	THEN '08. LE 680'
												WHEN [FICO] <= 700	THEN '09. LE 700'
												WHEN [FICO] <= 720	THEN '10. LE 720'
												WHEN [FICO] <= 740	THEN '11. LE 740'
												WHEN [FICO] <= 760	THEN '12. LE 760'
												WHEN [FICO] <= 780	THEN '13. LE 780'
												WHEN [FICO] > 780		THEN '14. 780+'
												END

In [22]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_9bf0168f2ef843769ed38a0ec4f1c748 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_9bf0168f2ef843769ed38a0ec4f1c748 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_9bf0168f2ef843769ed38a0ec4f1c748 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='27OeD3vEG6UgKXNZom2beclP2D4XFcgxedL-dSc7HtwL',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_9bf0168f2ef843769ed38a0ec4f1c748)

body = client_9bf0168f2ef843769ed38a0ec4f1c748.get_object(Bucket='testdeploy-donotdelete-pr-fkgaf6rwqj2ubj',Key='Test_fields_100821_results.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,HALF,DOWNNUM,YARDSTOGO,SCOREDIFF,TIMELEFT,DOWNNAME,TRUEFIELD,YARDSGAINED_Mean,AVG_OVER_TOGO,PLAYIDSTR,RUN_PASS,CONCEPT_8,CONCEPT_9,pred_TD,prob_TD_NO,prob_TD_YES,pred_FIRST,prob_FIRST_NO,prob_FIRST_YES
0,0,0,5,0,480,1st,35,6.360000,1.272000,100000020,RUN,2,0,N,0.989181,0.010819,N,0.562406,0.437594
1,0,0,9,-7,1050,1st,45,4.756098,0.528455,100000054,RUN,5,4,N,0.999278,0.000722,Y,0.439296,0.560704
2,0,2,4,7,600,3rd,5,8.164835,2.041209,200000049,PASS,4,9,N,0.929720,0.070280,Y,0.391968,0.608032
3,0,0,11,7,180,1st,45,6.216667,0.565152,200000041,PASS,4,1,N,0.999941,0.000059,N,0.676803,0.323197
4,0,0,1,0,60,1st,45,13.416667,13.416667,200000085,PASS,8,5,N,0.800383,0.199617,N,0.608495,0.391505


In [23]:
category = pd.cut(df.TIMELEFT,bins=[0,120,300,600,900,1200,2000],labels=['Two-minute','Five-minute','Ten-Minute','Fifteen-Minute','Early','Extra Time'])
df.insert(5,'Time_Segment',category)

In [24]:
df.columns

Index(['HALF', 'DOWNNUM', 'YARDSTOGO', 'SCOREDIFF', 'TIMELEFT', 'Time_Segment',
       'DOWNNAME', 'TRUEFIELD', 'YARDSGAINED_Mean', 'AVG_OVER_TOGO',
       'PLAYIDSTR', 'RUN_PASS', 'CONCEPT_8', 'CONCEPT_9', 'pred_TD',
       'prob_TD_NO', 'prob_TD_YES', 'pred_FIRST', 'prob_FIRST_NO',
       'prob_FIRST_YES'],
      dtype='object')

In [25]:
df.dtypes

HALF                   int64
DOWNNUM                int64
YARDSTOGO              int64
SCOREDIFF              int64
TIMELEFT               int64
Time_Segment        category
DOWNNAME              object
TRUEFIELD              int64
YARDSGAINED_Mean     float64
AVG_OVER_TOGO        float64
PLAYIDSTR              int64
RUN_PASS              object
CONCEPT_8              int64
CONCEPT_9              int64
pred_TD               object
prob_TD_NO           float64
prob_TD_YES          float64
pred_FIRST            object
prob_FIRST_NO        float64
prob_FIRST_YES       float64
dtype: object

In [ ]:
##Reduce fields
df_rev = pd.DataFrame(df, columns=['HALF', 'DOWN', 'YARDSTOGO', 'SCOREDIFF', 'TIMELEFT', 'DOWN_STRING','RUN_PASS',
       'CONCEPT_8', 'CONCEPT_9','YARDLINE','YARDSGAINED_Mean', 'AVG_OVER_TOGO', 'ISTOUCHDOWN', 'ISFIRSTDOWN'])

In [ ]:
## Re-Name columns
df_rev.columns = ['HALF', 'DOWNNUM', 'YARDSTOGO', 'SCOREDIFF', 'TIMELEFT', 'DOWNNAME','RUN_PASS','CONCEPT_8', 'CONCEPT_9',
                   'TRUEFIELD','YARDSGAINED_Mean', 'AVG_OVER_TOGO','ISTOUCHDOWN', 'ISFIRSTDOWN']

##RULE: Target columns must have same length as original

In [5]:
df_early = df[df['Time_Segment'].isin(['Early'])]
df_late = df[~df['Time_Segment'].isin(['Early'])]

In [6]:
df_early.Time_Segment.value_counts()

Early             6272
Extra Time           0
Fifteen-Minute       0
Ten-Minute           0
Five-minute          0
Two-minute           0
Name: Time_Segment, dtype: int64

In [7]:
df_late.Time_Segment.value_counts()

Two-minute        6272
Fifteen-Minute    4704
Ten-Minute        3136
Five-minute       3136
Extra Time           0
Early                0
Name: Time_Segment, dtype: int64

In [25]:
df([,1])

SyntaxError: invalid syntax (<ipython-input-25-2dafc9db982e>, line 1)

In [26]:
df.TIMELEFT.value_counts()

120     695
1140    687
840     682
480     681
180     674
60      673
30      671
300     668
720     667
90      663
900     659
1050    655
975     654
1200    637
600     634
Name: TIMELEFT, dtype: int64

In [27]:
df.groupby('Time_Segment').TIMELEFT.nunique()

Time_Segment
Two-minute        4
Five-minute       2
Ten-Minute        2
Fifteen-Minute    3
Early             4
Extra Time        0
Name: TIMELEFT, dtype: int64

In [28]:
df.Time_Segment.value_counts()

Two-minute        2702
Early             2633
Fifteen-Minute    2008
Five-minute       1342
Ten-Minute        1315
Extra Time           0
Name: Time_Segment, dtype: int64

In [65]:
df.YARDSTOGO.value_counts()

2     1037
1     1034
3     1025
5     1019
4      989
7      847
9      824
8      821
10     820
11     801
6      783
Name: YARDSTOGO, dtype: int64

FInd overall counts in your data

In [29]:
len(df)

10000

In [30]:
df.shape

(10000, 20)

In [31]:
df.TIMELEFT.shape

(10000,)

In [32]:
df.isnull().sum()

HALF                0
DOWNNUM             0
YARDSTOGO           0
SCOREDIFF           0
TIMELEFT            0
Time_Segment        0
DOWNNAME            0
TRUEFIELD           0
YARDSGAINED_Mean    0
AVG_OVER_TOGO       0
PLAYIDSTR           0
RUN_PASS            0
CONCEPT_8           0
CONCEPT_9           0
pred_TD             0
prob_TD_NO          0
prob_TD_YES         0
pred_FIRST          0
prob_FIRST_NO       0
prob_FIRST_YES      0
dtype: int64

In [13]:
df.TIMELEFT.isnull().sum()

0

In [33]:
df.pred_FIRST.value_counts()

Y    6240
N    3760
Name: pred_FIRST, dtype: int64

In [ ]:
df['logit'] = df.apply(lambda x: my_function(x['value_1'], x['value_2']), axis=1)

In [ ]:
a_lambda_function = lambda x: x*2 if x < 3 else x

In [60]:
import numpy as np

In [61]:
logit_idea = lambda x: np.log(0.00715/(1-0.00715)) if x < 3 else ((7.6781928 + -0.0173825 * df['prob_FIRST_YES']) + (-0.0013067 *df['prob_TD_YES']))

In [ ]:
np.where(a < 5, a, 10*a)

In [79]:
## Conversion using where 
df['logit_convert'] = np.where(df.RUN_PASS == 'RUN', np.log(0.00715/(1-0.00715)), ((7.6781928 + -0.0173825 * df['prob_FIRST_YES']) + (-0.0013067 *df['prob_TD_YES'])))

In [80]:
df.head()

,HALF,DOWNNUM,YARDSTOGO,SCOREDIFF,TIMELEFT,Time_Segment,DOWNNAME,TRUEFIELD,YARDSGAINED_Mean,AVG_OVER_TOGO,...,RUN_PASS,CONCEPT_8,CONCEPT_9,pred_TD,prob_TD_NO,prob_TD_YES,pred_FIRST,prob_FIRST_NO,prob_FIRST_YES,logit_convert
0,0,0,5,0,480,Ten-Minute,1st,35,6.360000,1.272000,...,RUN,2,0,N,0.989181,0.010819,N,0.562406,0.437594,-4.933467
1,0,0,9,-7,1050,Early,1st,45,4.756098,0.528455,...,RUN,5,4,N,0.999278,0.000722,Y,0.439296,0.560704,-4.933467
2,0,2,4,7,600,Ten-Minute,3rd,5,8.164835,2.041209,...,PASS,4,9,N,0.929720,0.070280,Y,0.391968,0.608032,7.667532
3,0,0,11,7,180,Five-minute,1st,45,6.216667,0.565152,...,PASS,4,1,N,0.999941,0.000059,N,0.676803,0.323197,7.672575
4,0,0,1,0,60,Two-minute,1st,45,13.416667,13.416667,...,PASS,8,5,N,0.800383,0.199617,N,0.608495,0.391505,7.671127


In [117]:
def calc(row):
#    for i in range (len(df)):
    if df['YARDSTOGO'].iloc[0] < 6:
        return np.log(0.00715/(1-0.00715))
#    elif row['two'] < 8:
#        return row['one']
    else:
        return (7.6781928 + -0.0173825 * df['prob_FIRST_YES']) + (-0.0013067 *df['prob_TD_YES'])

df['proba'] = df.apply(calc, axis=1)

In [120]:
def calc(x1, x2, x3):
    if x1 < 6:
        return np.log(0.00715/(1-0.00715))
    else:
        return (7.6781928 + -0.0173825 * x2) + (-0.0013067 * x3)
    
    

In [121]:
df['prob'] = df.apply(lambda x: calc(x['YARDSTOGO'], x['prob_FIRST_YES'], x['prob_TD_YES']), axis=1)

In [ ]:
data_df['aos_unsc_2100_velocity'] = data_df.apply(lambda x: velocity(x['aos_unsc_2000_input_count'], x['aos_unsc_2100_input_count']), axis=1)

In [122]:
df.head()

,HALF,DOWNNUM,YARDSTOGO,SCOREDIFF,TIMELEFT,Time_Segment,DOWNNAME,TRUEFIELD,YARDSGAINED_Mean,AVG_OVER_TOGO,...,CONCEPT_9,pred_TD,prob_TD_NO,prob_TD_YES,pred_FIRST,prob_FIRST_NO,prob_FIRST_YES,logit_convert,proba,prob
0,0,0,5,0,480,Ten-Minute,1st,35,6.360000,1.272000,...,0,N,0.989181,0.010819,N,0.562406,0.437594,-4.933467,7.670572,-4.933467
1,0,0,9,-7,1050,Early,1st,45,4.756098,0.528455,...,4,N,0.999278,0.000722,Y,0.439296,0.560704,-4.933467,7.670572,7.668445
2,0,2,4,7,600,Ten-Minute,3rd,5,8.164835,2.041209,...,9,N,0.929720,0.070280,Y,0.391968,0.608032,7.667532,7.670572,-4.933467
3,0,0,11,7,180,Five-minute,1st,45,6.216667,0.565152,...,1,N,0.999941,0.000059,N,0.676803,0.323197,7.672575,7.670572,7.672575
4,0,0,1,0,60,Two-minute,1st,45,13.416667,13.416667,...,5,N,0.800383,0.199617,N,0.608495,0.391505,7.671127,7.670572,-4.933467


In [123]:
df.shape

(10000, 23)

In [ ]:
def calc(row):
    if row['two'] > 8:
        return row['one'] * 100
    elif row['two'] < 8:
        return row['one']
    else:
        return row['one']**2

df['one'] = df.apply(calc, axis=1)